In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error

#dataset
data = pd.read_csv(r'G:\DNS Exfiltration Dataset\Mendely\mendeley_filtered.csv', header=None)

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349558 entries, 0 to 349557
Data columns (total 22 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       349558 non-null  object 
 1   1       349558 non-null  object 
 2   2       349558 non-null  float64
 3   3       349558 non-null  bool   
 4   4       349558 non-null  object 
 5   5       349558 non-null  int64  
 6   6       349558 non-null  int64  
 7   7       349558 non-null  int64  
 8   8       349558 non-null  int64  
 9   9       349558 non-null  float64
 10  10      349558 non-null  float64
 11  11      349558 non-null  float64
 12  12      349558 non-null  float64
 13  13      349558 non-null  float64
 14  14      349558 non-null  float64
 15  15      349558 non-null  float64
 16  16      349558 non-null  float64
 17  17      349558 non-null  float64
 18  18      349558 non-null  float64
 19  19      349558 non-null  float64
 20  20      349558 non-null  float64
 21  21      34

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,186.169.146.147,e5.sk,1.624440e+12,True,sebubx76xk4erpp3rwehoo3ubmbqeaqbaeaq.a.e.e5.sk,40,3,3,3,3.975071,...,0.125,0.0,2197.222222,2875.261022,48.2,53.370404,24.372977,0.0,3.691242,0.910175
1,186.169.146.147,e5.sk,1.624440e+12,True,4az3kiecotwu3okbtvfm7pdpcabqeaqbaeaq.a.e.e5.sk,40,3,5,3,4.146439,...,0.100,0.0,2348.444444,2779.448601,48.2,53.370404,22.803615,0.0,3.685581,0.906808
2,186.169.146.147,e5.sk,1.624440e+12,True,x3i2wbqsiucuviqyfaaoxz3lzybqeaqbaeaq.a.e.e5.sk,40,3,1,3,3.987326,...,0.075,0.0,2460.111111,2695.151964,51.8,51.228898,23.394454,0.0,3.884313,0.687639
3,186.169.127.58,e5.sk,1.624440e+12,True,ez2vzwchw3ce5m6wz6cw3nnc2ibqeaqbaeaq.a.e.e5.sk,40,3,1,3,3.893943,...,0.175,0.0,1799.222222,1935.781934,44.0,27.712813,27.170557,0.0,3.835620,0.663023
4,186.169.146.147,e5.sk,1.624440e+12,True,htm7xrligq2enc4lsjhkzdnd6mbqeaqbaeaq.a.e.e5.sk,40,3,3,3,4.371928,...,0.100,0.0,3105.444444,2782.422466,51.8,51.228898,18.533095,0.0,3.905225,0.700116


In [4]:
#Extract the feature used and the labels
x = data[[5]]
#x = data[[5]]

label = data[3]

print(label.shape)

(349558,)


In [5]:
#split data into training and testing
from sklearn.model_selection import train_test_split, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(x, label, test_size=0.35, random_state=42)


In [6]:
chunk = int(X_train.shape[0] * 0.8)
# print(chunk)
X_train = X_train.head(chunk)
y_train = y_train.head(chunk)
# X_train.head()
# y_train.head()
print(X_train.shape[0]+X_test.shape[0])

304115


In [7]:
#scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
#train the model and measure time

from sklearn.naive_bayes import GaussianNB
import time

model = GaussianNB()


scores = cross_val_score(model, X_train, y_train, cv=10)

start_time = time.time()
model.fit(X_train_scaled, y_train)
training_time = time.time() - start_time
print("training time:", training_time) 

training time: 0.009998321533203125


In [9]:
print(training_time)

0.009998321533203125


In [10]:
#predict using the trained model
from sklearn.metrics import accuracy_score

start_time = time.time()
y_pred = model.predict(X_test_scaled)
training_time = time.time() - start_time
print("testing time: ", training_time) 
accuracy = accuracy_score(y_test, y_pred)

print("accuracy: \n", accuracy)

print("10-fold Cross-Validation Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

testing time:  0.005997896194458008
accuracy: 
 0.9409379955208997
10-fold Cross-Validation Accuracy: 0.94 (+/- 0.00)


In [11]:
#create the confusion matrix
from sklearn.metrics import confusion_matrix

print("confusion matrix: \n", confusion_matrix(y_test, y_pred))


confusion matrix: 
 [[58320  2874]
 [ 4352 56800]]


In [12]:
#how many attacks the model detected
np.count_nonzero(y_pred == True)

59674

In [13]:
#how many attacks in the testing dataset
counter = 0

for i in y_test:
    if i == True:
        counter+=1
        
print(counter)

61152
